In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery
from matplotlib.ticker import PercentFormatter


from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_spending_by_mcg = '''SELECT time_period_value, mcg, spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'Online'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS'
and mcc != 'All'
GROUP BY mcg, time_period_value, spend
ORDER BY time_period_value, spend DESC'''

mcg_by_quarter = bq.read_bq_table_sql(client, UK_spending_by_mcg)

mcg_by_quarter['year'] = mcg_by_quarter['time_period_value'].str[:4]

# Convert the 'year' column to numeric
mcg_by_quarter['year'] = pd.to_numeric(mcg_by_quarter['year'], errors='coerce')

mcg_by_quarter.head()

In [ ]:

# Filter data for 2019 and 2024
df_filtered = mcg_by_quarter[mcg_by_quarter['year'].isin([2019, 2024])]

# Group by year and category, summing the spend
grouped = df_filtered.groupby(['year', 'mcg'])['spend'].sum().reset_index()

# Pivot the data to have years as columns
pivot_df = grouped.pivot(index='mcg', columns='year', values='spend').fillna(0)

# Calculate percentage of total spend for each year
pivot_df[2019] = pivot_df[2019] / pivot_df[2019].sum()
pivot_df[2024] = pivot_df[2024] / pivot_df[2024].sum()

# Sort categories by 2024 spend
pivot_df = pivot_df.sort_values(by=2024, ascending=False)

# Plotting
categories = pivot_df.index.tolist()
spend_2024 = pivot_df[2024].values
spend_2019 = pivot_df[2019].values

x = np.arange(len(categories))
width = 0.6

fig, ax = plt.subplots(figsize=(14, 8))

# Bar for 2024
bars = ax.bar(x, spend_2024, width, label='2024', color='orange')

# Diamond markers for 2019 overlapping the bars
ax.scatter(x, spend_2019, color='blue', marker='D', label='2019')

# Labels and formatting
ax.set_xlabel('Categories')
ax.set_ylabel('Share of Spend (%)')
ax.set_title('Share of Online Spend by Merchant Category Group, 2019 vs 2024')
ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=90)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))  # Format y-axis as whole percentages
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(pivot_df)

In [ ]:
print(grouped)

In [ ]:
grouped.to_csv('grouped.csv')

In [ ]:
# Change in each mcg from 2019 to 2024
pivot_df = grouped.pivot(index='mcg', columns='year', values='spend').fillna(0)

#Calculate absolute and percent change
pivot_df['absolute_change'] = pivot_df[2024] - pivot_df[2019]
pivot_df['percent_change'] = (pivot_df['absolute_change'] / pivot_df[2019]) * 100

pivot_df

In [ ]:
# Total spend in each MCG to calcualte the online ratios
# Summarise the data by mcg
total_spending_by_mcg = '''SELECT time_period_value, mcg, spend AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter'
and time_period_value != '2025Q1'  
and merchant_channel = 'All'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS'
and mcc != 'All'
GROUP BY mcg, time_period_value, spend
ORDER BY time_period_value, spend DESC'''

total_mcg_by_quarter = bq.read_bq_table_sql(client, total_spending_by_mcg)

total_mcg_by_quarter['year'] = total_mcg_by_quarter['time_period_value'].str[:4]

# Convert the 'year' column to numeric
total_mcg_by_quarter['year'] = pd.to_numeric(total_mcg_by_quarter['year'], errors='coerce')

total_mcg_by_quarter.head()

In [ ]:
total_df_filtered = total_mcg_by_quarter[total_mcg_by_quarter['year'].isin([2019, 2024])]
total_grouped = total_df_filtered.groupby(['year', 'mcg'])['total_spend'].sum().reset_index()

merged_df = pd.merge(total_grouped, grouped, on=["mcg", "year"])
merged_df["Online %"] = merged_df["spend"] / merged_df["total_spend"] * 100
merged_df

In [ ]:
# Change in online ratio for each mcg from 2019 to 2024
pivot_df = merged_df.pivot(index='mcg', columns='year', values='spend').fillna(0)

#Calculate absolute and percent change
pivot_df['absolute_change'] = pivot_df[2024] - pivot_df[2019]
pivot_df['percent_change'] = (pivot_df['absolute_change'] / pivot_df[2019]) * 100

pivot_df